In [83]:
import pandas as pd
import os
import json
import glob
import sqlite3

In [90]:
#ouvert les paramètres décrits dans le fichier JSON
with open('../parameters.json') as json_data:
    params = json.load(json_data)


In [91]:
params

{'DATABASE': {'GAS_DB': 'gas.db'},
 'DIRECTORIES': {'DATA_DIR': '../../data', 'TMP_DIR': '../../tmp'},
 'FILES': {'PRICE_FILE': 'prices.p',
  'SERVICE_FILE': 'Services.p',
  'STATION_FILE': 'Stations.p'}}

In [92]:
#utilisation des paramètres pour créer un DATA et TMP
DATA_DIR = params['DIRECTORIES']['DATA_DIR']
TMP_DIR = params['DIRECTORIES']['TMP_DIR']
PRICE_FILE = params['FILES']['PRICE_FILE']
STATION_FILE = params['FILES']['STATION_FILE']
SERVICE_FILE = params['FILES']['SERVICE_FILE']
GAS_DB = os.path.join(DATA_DIR, params['DATABASE']['GAS_DB'])

In [41]:
gas_files = glob.glob(os.path.join(DATA_DIR,"Prix*.zip"))

In [76]:
stations_file = 'Stations2016.zip'
service_file = 'Services2016.zip'

In [15]:
#ouverture de la table
df2014 = pd.read_csv(os.path.join(DATA_DIR, gas_files[0]), sep=";", header=None, dtype = {1 : str})
df2014.columns = ['id_stations', 
                  'code_postal',
                  'type_station', 
                  'latitude',
                  'longitude',
                  'date_releve',
                  'type_carburant', 
                 'libelle_carburant',
                  'prix']

In [28]:
df2014.head()

,id_stations,code_postal,type_station,latitude,longitude,date_releve,type_carburant,libelle_carburant,prix
0,1000001,01000,R,4620114.0,519791.0,2014-01-02 11:08:03,1.0,Gazole,1304.0
1,1000001,01000,R,4620114.0,519791.0,2014-01-04 09:54:03,1.0,Gazole,1304.0
2,1000001,01000,R,4620114.0,519791.0,2014-01-05 10:27:09,1.0,Gazole,1304.0
3,1000001,01000,R,4620114.0,519791.0,2014-01-06 09:07:51,1.0,Gazole,1304.0
4,1000001,01000,R,4620114.0,519791.0,2014-01-07 09:23:56,1.0,Gazole,1304.0


In [45]:
%%timeit

#concatenation des années

dfall = pd.DataFrame()

for year in gas_files:
    df = pd.read_csv(year, sep=";", header=None, dtype = {1 : str})

    df.columns = ['id_stations', 
                  'code_postal',
                  'type_station', 
                  'latitude',
                  'longitude',
                  'date_releve',
                  'type_carburant', 
                 'libelle_carburant',
                  'prix']
                     
    dfall = pd.concat([dfall, df], axis=0)
dfall.shap    


In [53]:
assert dfall.shape[1] == 9, "Pas le bon nombre de colonnes"

In [39]:
#donne la liste des fichiers
import glob

gas_files = glob.glob(os.path.join(DATA_DIR,"Prix*.zip"))

In [40]:
gas_files

['../../data\\Prix2007.zip',
 '../../data\\Prix2008.zip',
 '../../data\\Prix2009.zip',
 '../../data\\Prix2010.zip',
 '../../data\\Prix2011.zip',
 '../../data\\Prix2012.zip',
 '../../data\\Prix2013.zip',
 '../../data\\Prix2014.zip',
 '../../data\\Prix2015.zip',
 '../../data\\Prix2016.zip']

In [77]:
dfstations = pd.read_csv(os.path.join(DATA_DIR, stations_file),sep ="|", encoding ='utf-8', header = None, dtype = {1 : str})

dfstations.columns = ['id_station', 'code_postal', 'type_de_station', 'latitude', 'longitude', 'adresse', 'ville']

In [65]:
dfstations.head()

,id_station,code_postal,type_de_station,latitude,longitude,adresse,ville
0,1000001,01000,R,4.620114e+06,519791.000000,596 AVENUE DE TREVOUX,SAINT-DENIS-LèS-BOURG
1,1000002,01000,R,4.621842e+06,522767.000000,16 Avenue de Marboz,BOURG-EN-BRESSE
2,1000004,01000,R,4.618836e+06,524447.000000,20 Avenue du Maréchal Juin,Bourg-en-Bresse
3,1000005,01000,R,4.620094e+06,519942.024022,642 Avenue de Trévoux,SAINT-DENIS-LèS-BOURG
4,1000006,01000,R,4.620754e+06,523758.000000,1 Boulevard John Kennedy,BOURG-EN-BRESSE


In [78]:
dfservices = pd.read_csv(os.path.join(DATA_DIR, service_file),sep ="|", encoding ='utf-8', header = None, dtype = {1 : str})

dfservices.columns = ['id_station', 'code_postal', 'type_de_station', 'latitude', 'longitude', 'services']

In [64]:
dfservices.head()

,id_station,code_postal,type_de_station,latitude,longitude,services
0,1000001,01000,R,4.620114e+06,519791.000000,"Station de gonflage,Vente de gaz domestique,Au..."
1,1000002,01000,R,4.621842e+06,522767.000000,"Vente de gaz domestique,Automate CB"
2,1000004,01000,R,4.618836e+06,524447.000000,"Carburant qualité supérieure,GPL,Toilettes pub..."
3,1000005,01000,R,4.620094e+06,519942.024022,"Carburant qualité supérieure,Station de lavage..."
4,1000006,01000,R,4.620754e+06,523758.000000,Vente de gaz domestique


In [81]:
dfall.to_pickle(os.path.join(DATA_DIR, PRICE_FILE))
dfstations.to_pickle(os.path.join(DATA_DIR, STATION_FILE))
dfservices.to_pickle(os.path.join(DATA_DIR, SERVICE_FILE))

In [82]:
# test le temps de lecture
%%timeit
testdf = pd.read_pickle(os.path.join(DATA_DIR, PRICE_FILE))

1 loop, best of 3: 5.34 s per loop


In [107]:
cnx = sqlite3.connect(GAS_DB)
dfall.to_sql('prices', cnx ,index = False, if_exists = "replace")

In [108]:
dfservices.to_sql('Services', cnx, index = False, if_exists = "replace")
dfstations.to_sql('Stations', cnx, index = False, if_exists = "replace")

OperationalError: table "Services" already exists

In [103]:
pd.read_sql_query('select * from prices where code_postal ="13001"', cnx)

,id_stations,code_postal,type_station,latitude,longitude,date_releve,type_carburant,libelle_carburant,prix
0,13001001.0,13001,R,4330446.0,538957.0,2007-01-11 10:27:23,1.0,Gazole,1039.0
1,13001001.0,13001,R,4330446.0,538957.0,2007-01-15 08:10:34,1.0,Gazole,1024.0
2,13001001.0,13001,R,4330446.0,538957.0,2007-01-18 07:49:51.425658,1.0,Gazole,1014.0
3,13001001.0,13001,R,4330446.0,538957.0,2007-01-29 08:45:27.511952,1.0,Gazole,1018.0
4,13001001.0,13001,R,4330446.0,538957.0,2007-02-07 06:00:00,1.0,Gazole,1027.0
5,13001001.0,13001,R,4330446.0,538957.0,2007-02-22 06:00:00,1.0,Gazole,1018.0
6,13001001.0,13001,R,4330446.0,538957.0,2007-02-27 06:00:00,1.0,Gazole,1029.0
7,13001001.0,13001,R,4330446.0,538957.0,2007-03-02 06:00:00,1.0,Gazole,1039.0
8,13001001.0,13001,R,4330446.0,538957.0,2007-03-15 06:00:00,1.0,Gazole,1044.0
9,13001001.0,13001,R,4330446.0,538957.0,2007-03-24 06:00:00,1.0,Gazole,1040.0
